In [1]:
import speech_recognition as sr
from pywebio import start_server
from pywebio.input import input, textarea
from pywebio.output import put_text, put_html, put_buttons
from pywebio.session import defer_call

In [2]:
def chatbot_app():
    put_html("<div style='background-color: white; padding: 20px;'> <h1>Emotion Depicter</h1> </div>")
    
    def handle_input(user_input):
        put_text("User:", user_input)

        # Here you can add the logic to process the user's input and generate a response
        response = "Placeholder response"

        put_text("Chatbot:", response)
    
    # Function to handle text input
    def handle_text_input():
        user_input = input("User:", placeholder="Type here...")
        handle_input(user_input)
    
    # Function to handle speech input
    def handle_speech_input():
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            put_text("Speak", "Speak something...")
            audio = recognizer.listen(source)
        
        try:
            user_input = recognizer.recognize_google(audio)
            handle_input(user_input)
        except sr.UnknownValueError:
            put_text("Sorry, I could not understand what you said.")
        except sr.RequestError as e:
            put_text("Sorry, I could not request results from Google Speech Recognition service; {0}".format(e))

    put_buttons(['Please Write', '🎤'], onclick=[handle_text_input, handle_speech_input])

In [3]:
if __name__ == "__main__":
    start_server(chatbot_app, port=8088)

Running on all addresses.
Use http://172.16.145.192:8088/ to access the application


RuntimeError: This event loop is already running

result2:
{   'alternative': [   {   'transcript': 'hey I am feeling very happy I got '
                                         'great marks'}],
    'final': True}
